In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=ebbaa28889eb850738269f56ee1b4d347a70a6928dde82fcb2a8fe390a033397
  Stored in directory: c:\users\brand\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV

from matplotlib import style
style.use("fivethirtyeight")

from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

In [9]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [11]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
X= selected_features
X.head()
print(X.shape)

(6991, 8)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [12]:
y= df['koi_disposition'].values.reshape(-1,1)
print(y.shape)

(6991, 1)


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)
print("Distribution of classes within data")
print("_"*30)
print(pd.Series(y).value_counts())

print("Distribution of classes within training set")
print("_"*30)
print(pd.Series(y_train).value_counts())

In [14]:
X_train.head()

koi_fpflag_nt  koi_period  koi_time0bk  koi_slogg  koi_srad         ra  \
6122              0    6.768901   133.077240      4.327     1.125  294.40472   
6370              0    0.733726   132.020050      4.578     0.797  284.50391   
2879              1    7.652707   134.460380      4.481     0.963  295.50211   
107               0    7.953547   174.662240      4.536     0.779  291.15878   
29                0    4.959319   172.258529      4.359     1.082  292.16705   

            dec  koi_kepmag  
6122  39.351681      14.725  
6370  42.463860      15.770  
2879  38.983540      13.099  
107   40.750271      15.660  
29    48.727589      15.263

# Create a logistic Regression Model



ValueError: could not convert string to float: 'CANDIDATE'

# Fit (train) model using the training data



# Validate the model using the test data



# Generate a new data point

# Predict the class of the new data point

#  View the rows with incorrect predictions

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)